In [ ]:
#Build a Funnel From a Single Table
SELECT question_text, COUNT(DISTINCT user_id)
FROM survey_responses
GROUP BY question_text;

In [ ]:
#Compare Funnels For A/B Tests
SELECT modal_text,
  COUNT(DISTINCT CASE
    WHEN ab_group = 'control' THEN user_id
    END) AS 'control_clicks',
    COUNT(DISTINCT CASE
    WHEN ab_group = 'variant' THEN user_id
    END) AS 'variant_clicks'
FROM onboarding_modals
GROUP BY 1
ORDER BY 1;

In [ ]:
#Build a Funnel from Multiple Tables 1
SELECT *
FROM browse
LIMIT 5;
 
SELECT *
FROM checkout
LIMIT 5;
 
SELECT *
FROM purchase
LIMIT 5;

In [ ]:
#Build a Funnel from Multiple Tables 2
SELECT DISTINCT b.browse_date,
b.user_id,
c.user_id IS NOT NULL AS 'is_checkout',
p.user_id IS NOT NULL AS 'is_purchase'
FROM browse b
LEFT JOIN checkout c
ON b.user_id = c.user_id
LEFT JOIN purchase p
ON b.user_id = p.user_id
LIMIT 50;

In [ ]:
#Build a Funnel from Multiple Tables 3
WITH funnels AS (
  SELECT DISTINCT b.browse_date,
     b.user_id,
     c.user_id IS NOT NULL AS 'is_checkout',
     p.user_id IS NOT NULL AS 'is_purchase'
  FROM browse AS 'b'
  LEFT JOIN checkout AS 'c'
    ON c.user_id = b.user_id
  LEFT JOIN purchase AS 'p'
    ON p.user_id = c.user_id)
SELECT COUNT(*) AS 'num_browse',
SUM(is_checkout) AS 'num_checkout',
SUM(is_purchase) AS 'num_purchase',
1.0 * SUM(is_checkout) / COUNT(user_id) AS 'percent_browse',
1.0 * SUM(is_purchase) / SUM(is_checkout) AS 'percent_checkout'
FROM funnels;

In [ ]:
#Build a Funnel from Multiple Tables 4
WITH funnels AS (
  SELECT DISTINCT b.browse_date,
     b.user_id,
     c.user_id IS NOT NULL AS 'is_checkout',
     p.user_id IS NOT NULL AS 'is_purchase'
  FROM browse AS 'b'
  LEFT JOIN checkout AS 'c'
    ON c.user_id = b.user_id
  LEFT JOIN purchase AS 'p'
    ON p.user_id = c.user_id)
SELECT funnels.browse_date,
   COUNT(*) AS 'num_browse',
   SUM(is_checkout) AS 'num_checkout',
   SUM(is_purchase) AS 'num_purchase',
   1.0 * SUM(is_checkout) / COUNT(user_id) AS 'browse_to_checkout',
   1.0 * SUM(is_purchase) / SUM(is_checkout) AS 'checkout_to_purchase'
FROM funnels
GROUP BY 1
ORDER BY 1;
#Conversion from checkout to purchase increases from 80% on 12/20 to 94% on 12/23!